# SQLAlchemy Object Relational Mapping

https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.4'

In [2]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://localhost/wimbledon', echo=True)

# Declare Mapping 

In [3]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String

class Person(Base):
    __tablename__ = 'people'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    association = Column(Integer) # SHOULD BE FOREIGN KEY TO AN ASSOCIATION TABLE
    
    def __repr__(self):
        return "<Person(name='{:s}', association='{:d}')>".format(self.name, self.association)

# Create Schema

In [5]:
Person.__table__

Table('people', MetaData(bind=None), Column('id', Integer(), table=<people>, primary_key=True, nullable=False), Column('name', String(), table=<people>), Column('association', Integer(), table=<people>), schema=None)

In [6]:
Base.metadata.create_all(engine)

2019-07-31 17:46:22,054 INFO sqlalchemy.engine.base.Engine select version()
2019-07-31 17:46:22,054 INFO sqlalchemy.engine.base.Engine {}
2019-07-31 17:46:22,056 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-07-31 17:46:22,057 INFO sqlalchemy.engine.base.Engine {}
2019-07-31 17:46:22,059 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-31 17:46:22,060 INFO sqlalchemy.engine.base.Engine {}
2019-07-31 17:46:22,061 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-31 17:46:22,062 INFO sqlalchemy.engine.base.Engine {}
2019-07-31 17:46:22,063 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-07-31 17:46:22,064 INFO sqlalchemy.engine.base.Engine {}
2019-07-31 17:46:22,067 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

# Create Instance 

In [7]:
jack_person = Person(name='Jack Roberts', association=1)
print(jack_person)

<Person(name='Jack Roberts', association='1')>


# Create Session 

In [8]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

In [9]:
session = Session()

# Add Update

In [10]:
session.add(jack_person)

In [11]:
my_person = session.query(Person).filter_by(name='Jack Roberts').first()
my_person

2019-07-31 17:46:22,102 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-31 17:46:22,104 INFO sqlalchemy.engine.base.Engine INSERT INTO people (name, association) VALUES (%(name)s, %(association)s) RETURNING people.id
2019-07-31 17:46:22,105 INFO sqlalchemy.engine.base.Engine {'name': 'Jack Roberts', 'association': 1}
2019-07-31 17:46:22,109 INFO sqlalchemy.engine.base.Engine SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people 
WHERE people.name = %(name_1)s 
 LIMIT %(param_1)s
2019-07-31 17:46:22,110 INFO sqlalchemy.engine.base.Engine {'name_1': 'Jack Roberts', 'param_1': 1}


<Person(name='Jack Roberts', association='1')>

In [12]:
session.add_all([
    Person(name='Oliver Strickson', association=1),
    Person(name='James Geddes', association=2),
    Person(name='Ed C', association=3),
    Person(name='Kasra H', association=1),
    Person(name='Eric Daub', association=4)
])

In [13]:
jack_person.name = 'Jack RRRRRRRoberts'

In [14]:
session.dirty

IdentitySet([<Person(name='Jack RRRRRRRoberts', association='1')>])

In [15]:
session.new

IdentitySet([<Person(name='Oliver Strickson', association='1')>, <Person(name='James Geddes', association='2')>, <Person(name='Ed C', association='3')>, <Person(name='Kasra H', association='1')>, <Person(name='Eric Daub', association='4')>])

In [16]:
session.commit()

2019-07-31 17:46:22,153 INFO sqlalchemy.engine.base.Engine UPDATE people SET name=%(name)s WHERE people.id = %(people_id)s
2019-07-31 17:46:22,153 INFO sqlalchemy.engine.base.Engine {'name': 'Jack RRRRRRRoberts', 'people_id': 1}
2019-07-31 17:46:22,155 INFO sqlalchemy.engine.base.Engine INSERT INTO people (name, association) VALUES (%(name)s, %(association)s) RETURNING people.id
2019-07-31 17:46:22,156 INFO sqlalchemy.engine.base.Engine {'name': 'Oliver Strickson', 'association': 1}
2019-07-31 17:46:22,158 INFO sqlalchemy.engine.base.Engine INSERT INTO people (name, association) VALUES (%(name)s, %(association)s) RETURNING people.id
2019-07-31 17:46:22,159 INFO sqlalchemy.engine.base.Engine {'name': 'James Geddes', 'association': 2}
2019-07-31 17:46:22,160 INFO sqlalchemy.engine.base.Engine INSERT INTO people (name, association) VALUES (%(name)s, %(association)s) RETURNING people.id
2019-07-31 17:46:22,161 INFO sqlalchemy.engine.base.Engine {'name': 'Ed C', 'association': 3}
2019-07-31

In [17]:
jack_person.id

2019-07-31 17:46:22,175 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-31 17:46:22,176 INFO sqlalchemy.engine.base.Engine SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people 
WHERE people.id = %(param_1)s
2019-07-31 17:46:22,177 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


1

# Roll back

In [18]:
jack_person.name = 'JACKOOOOOO'

In [19]:
fake_person = Person(name='fakeuser', association=342)
session.add(fake_person)

In [20]:
session.query(Person).filter(Person.name.in_(['JACKOOOOOO', 'fakeuser'])).all()

2019-07-31 17:46:22,199 INFO sqlalchemy.engine.base.Engine UPDATE people SET name=%(name)s WHERE people.id = %(people_id)s
2019-07-31 17:46:22,200 INFO sqlalchemy.engine.base.Engine {'name': 'JACKOOOOOO', 'people_id': 1}
2019-07-31 17:46:22,201 INFO sqlalchemy.engine.base.Engine INSERT INTO people (name, association) VALUES (%(name)s, %(association)s) RETURNING people.id
2019-07-31 17:46:22,202 INFO sqlalchemy.engine.base.Engine {'name': 'fakeuser', 'association': 342}
2019-07-31 17:46:22,204 INFO sqlalchemy.engine.base.Engine SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people 
WHERE people.name IN (%(name_1)s, %(name_2)s)
2019-07-31 17:46:22,205 INFO sqlalchemy.engine.base.Engine {'name_1': 'JACKOOOOOO', 'name_2': 'fakeuser'}


[<Person(name='JACKOOOOOO', association='1')>,
 <Person(name='fakeuser', association='342')>]

In [21]:
session.rollback()

2019-07-31 17:46:22,214 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [22]:
jack_person.name

2019-07-31 17:46:22,221 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-31 17:46:22,222 INFO sqlalchemy.engine.base.Engine SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people 
WHERE people.id = %(param_1)s
2019-07-31 17:46:22,223 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


'Jack RRRRRRRoberts'

In [23]:
fake_person in session

False

# Query 

In [24]:
for instance in session.query(Person).order_by(Person.id):
    print(instance)

2019-07-31 17:46:22,237 INFO sqlalchemy.engine.base.Engine SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people ORDER BY people.id
2019-07-31 17:46:22,238 INFO sqlalchemy.engine.base.Engine {}
<Person(name='Jack RRRRRRRoberts', association='1')>
<Person(name='Oliver Strickson', association='1')>
<Person(name='James Geddes', association='2')>
<Person(name='Ed C', association='3')>
<Person(name='Kasra H', association='1')>
<Person(name='Eric Daub', association='4')>


In [25]:
for name, association in session.query(Person.name, Person.association):
    print(name, association)

2019-07-31 17:46:22,245 INFO sqlalchemy.engine.base.Engine SELECT people.name AS people_name, people.association AS people_association 
FROM people
2019-07-31 17:46:22,246 INFO sqlalchemy.engine.base.Engine {}
Jack RRRRRRRoberts 1
Oliver Strickson 1
James Geddes 2
Ed C 3
Kasra H 1
Eric Daub 4


In [26]:
for row in session.query(Person, Person.name).all():
    print(row.Person, row.name)

2019-07-31 17:46:22,255 INFO sqlalchemy.engine.base.Engine SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people
2019-07-31 17:46:22,256 INFO sqlalchemy.engine.base.Engine {}
<Person(name='Jack RRRRRRRoberts', association='1')> Jack RRRRRRRoberts
<Person(name='Oliver Strickson', association='1')> Oliver Strickson
<Person(name='James Geddes', association='2')> James Geddes
<Person(name='Ed C', association='3')> Ed C
<Person(name='Kasra H', association='1')> Kasra H
<Person(name='Eric Daub', association='4')> Eric Daub


In [27]:
for row in session.query(Person.name.label('name_label')).all():
    print(row.name_label)

2019-07-31 17:46:22,262 INFO sqlalchemy.engine.base.Engine SELECT people.name AS name_label 
FROM people
2019-07-31 17:46:22,263 INFO sqlalchemy.engine.base.Engine {}
Jack RRRRRRRoberts
Oliver Strickson
James Geddes
Ed C
Kasra H
Eric Daub


In [28]:
for name, in session.query(Person.name).filter(Person.name=='Eric Daub'):
    print(name)

2019-07-31 17:46:22,271 INFO sqlalchemy.engine.base.Engine SELECT people.name AS people_name 
FROM people 
WHERE people.name = %(name_1)s
2019-07-31 17:46:22,271 INFO sqlalchemy.engine.base.Engine {'name_1': 'Eric Daub'}
Eric Daub


In [29]:
for name, in session.query(Person.name).filter(Person.name.like('%James%')):
    print(name)

2019-07-31 17:46:22,279 INFO sqlalchemy.engine.base.Engine SELECT people.name AS people_name 
FROM people 
WHERE people.name LIKE %(name_1)s
2019-07-31 17:46:22,280 INFO sqlalchemy.engine.base.Engine {'name_1': '%James%'}
James Geddes


In [30]:
session.query(Person).filter(Person.name.like('J%')).count()

2019-07-31 17:46:22,287 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT people.id AS people_id, people.name AS people_name, people.association AS people_association 
FROM people 
WHERE people.name LIKE %(name_1)s) AS anon_1
2019-07-31 17:46:22,288 INFO sqlalchemy.engine.base.Engine {'name_1': 'J%'}


2

# Relationships 

In [31]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy import Date

class Assignment(Base):
    __tablename__ = 'assignments'
    id = Column(Integer, primary_key=True)
    start_date = Column(Date)
    end_date = Column(Date)
    person = Column(Integer, ForeignKey('people.id'))
    
